In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision as tv

import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from tqdm import tqdm

In [233]:
class DatasetToClass(torch.utils.data.Dataset):
    def __init__(self, path_dirs):
        super().__init__()
        
        self.path_dirs = []
        for i in path_dirs:
            self.path_dirs.append(i)
        
        self.class_borders=[]
        self.class_borders.append(0)
        #print(sorted(os.listdir(path_dirs[0])))
        self.dir_list=[]
        for i in range(1, len(path_dirs)+1):
            self.dir_list.append(sorted(os.listdir(path_dirs[i-1])))
            self.class_borders.append(len(self.dir_list[i-1])+ self.class_borders[i-1])
        #print(self.class_borders)
    
    def __len__(self):
        length = 0
        for i in self.dir_list:
            length += len(i)
        return length
    
    def __getitem__(self,idx):
        class_id=0
        for i in range(1,11):
            if idx >= self.class_borders[i-1] and idx < self.class_borders[i]:
                class_id = i-1
                break
        img_path = os.path.join(self.path_dirs[class_id], self.dir_list[class_id][self.class_borders[class_id]-idx])

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        img=img.astype(np.float32)
        img = img/255.0
        img = cv2.resize(img,(128,128), interpolation = cv2.INTER_AREA)
        img = img.transpose((2,0,1))
        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)
        #return img
        return {'img': t_img, 'label':t_class_id}        

In [234]:
train_ds = DatasetToClass([
    '/home/andrey/GraduateWork/IMG_CLASSES/1. Eczema 1677',
    '/home/andrey/GraduateWork/IMG_CLASSES/2. Melanoma 15.75k',
    '/home/andrey/GraduateWork/IMG_CLASSES/3. Atopic Dermatitis - 1.25k',
    '/home/andrey/GraduateWork/IMG_CLASSES/4. Basal Cell Carcinoma (BCC) 3323',
    '/home/andrey/GraduateWork/IMG_CLASSES/5. Melanocytic Nevi (NV) - 7970',
    '/home/andrey/GraduateWork/IMG_CLASSES/6. Benign Keratosis-like Lesions (BKL) 2624',
    '/home/andrey/GraduateWork/IMG_CLASSES/7. Psoriasis pictures Lichen Planus and related diseases - 2k',
    '/home/andrey/GraduateWork/IMG_CLASSES/8. Seborrheic Keratoses and other Benign Tumors - 1.8k',
    '/home/andrey/GraduateWork/IMG_CLASSES/9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k',
    '/home/andrey/GraduateWork/IMG_CLASSES/10. Warts Molluscum and other Viral Infections - 2103'
])

test_ds = DatasetToClass([
    '/home/andrey/GraduateWork/IMG_CLASSES/1. TEczema 1677',
    '/home/andrey/GraduateWork/IMG_CLASSES/2. TMelanoma 15.75k',
    '/home/andrey/GraduateWork/IMG_CLASSES/3. TAtopic Dermatitis - 1.25k',
    '/home/andrey/GraduateWork/IMG_CLASSES/4. TBasal Cell Carcinoma (BCC) 3323',
    '/home/andrey/GraduateWork/IMG_CLASSES/5. TMelanocytic Nevi (NV) - 7970',
    '/home/andrey/GraduateWork/IMG_CLASSES/6. TBenign Keratosis-like Lesions (BKL) 2624',
    '/home/andrey/GraduateWork/IMG_CLASSES/7. TPsoriasis pictures Lichen Planus and related diseases - 2k',
    '/home/andrey/GraduateWork/IMG_CLASSES/8. TSeborrheic Keratoses and other Benign Tumors - 1.8k',
    '/home/andrey/GraduateWork/IMG_CLASSES/9. TTinea Ringworm Candidiasis and other Fungal Infections - 1.7k',
    '/home/andrey/GraduateWork/IMG_CLASSES/10. TWarts Molluscum and other Viral Infections - 2103'
])

In [235]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_ds, shuffle = True, batch_size = batch_size, num_workers=1, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_ds, shuffle = True, batch_size = batch_size, num_workers=1, drop_last=False)

In [257]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.LeakyReLU(0.2)
        self.maxpool = nn.MaxPool2d(2,2)
        self.conv0 = nn.Conv2d(3, 32 , 3, stride=1, padding=0)
        self.conv1 = nn.Conv2d(32, 32 , 3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 64 , 3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(64, 128 , 3, stride=1, padding=0)

        self.adaptivepool = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(128,32)
        self.linear2 = nn.Linear(32,10)

    
    def forward(self, x):
        out = self.conv0(x)
        out = self.act(out)
        out = self.maxpool(out)
        out = self.conv1(out)
        out = self.act(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = self.act(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.act(out)
        
        out = self.adaptivepool(out)
        out = self.flatten(out)
        out = self.linear1(out)
        out = self.act(out)
        out = self.linear2(out)
        return out    

In [258]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [259]:
model = ConvNet()

In [260]:
model

ConvNet(
  (act): LeakyReLU(negative_slope=0.2)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (adaptivepool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=128, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=10, bias=True)
)

In [261]:
count_parameters(model)

106954

In [262]:
for sample in train_loader:
    img = sample['img']
    label = sample['label']
    model(img)
    break

In [263]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [264]:
def accuracy(pred, label):
    answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1) 
    return answer.mean()

In [265]:
#device = 'cuda' # if torch.cuda.is_available() else 'cpu'
#model = model.to(device)
#loss_fn = loss_fn.to(device)

In [266]:
#use_amp = True
#scaler = torch.cuda.amp.GradScaler()

In [267]:
#torch.backends.cudnn.benchmark = True
#torch.backends.cudnn.deterministic = False

In [268]:
len(train_loader)

1528

In [269]:
epochs = 10

for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        img, label = sample['img'], sample['label']
        optimizer.zero_grad()
        #img = img.to(device)
        #label = label.to(device)
        
        label = F.one_hot(label, 10).float()
        pred = model(img)
        
        loss = loss_fn(pred, label)

        loss.backward()
        loss_item = loss.item()
        loss_val += loss_item

        optimizer.step()

        acc_current = accuracy(pred, label)
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
    print(loss_val/len(train_loader))
    print(acc_val/len(train_loader))

  0%|                                                                                          | 0/1528 [00:00<?, ?it/s]/tmp/ipykernel_893/156703249.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
100%|███████████████████████████████████████████████████████████████████████████████| 1528/1528 [05:58<00:00,  4.27it/s]


1.756192687840362
0.37381380890052357


100%|███████████████████████████████████████████████████████████████████████████████| 1528/1528 [05:29<00:00,  4.63it/s]


1.4971238950082144
0.4565608638743455


100%|███████████████████████████████████████████████████████████████████████████████| 1528/1528 [05:07<00:00,  4.97it/s]


...

1.0520027937728385
0.5968586387434555


100%|███████████████████████████████████████████████████████████████████████████████| 1528/1528 [05:05<00:00,  5.01it/s]


1.0227092187169022
0.6085160340314136


  3%|██▏                                                                              | 41/1528 [00:09<05:53,  4.21it/s]


KeyboardInterrupt: 

In [ ]:
loss_val = 0
acc_val = 0
for sample in (pbar := tqdm(test_loader)):
    with torch.no_grad():
        img, label = sample['img'], sample['label']

        label = F.one_hot(label, 2).float()
        pred = model(img)

        loss = loss_fn(pred, label)
        loss_item = loss.item()
        loss_val += loss_item

        acc_current = accuracy(pred, label)
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
print(loss_val/len(train_loader))
print(acc_val/len(train_loader))